In [100]:
# Import the f distribution from scipy.stats
from scipy.stats import f, t
import numpy as np
from scipy.optimize import newton, minimize, fmin_tnc
class LinearModel():
    """The Linear Model Class is the parent class to all linear models."""
    
    def __init__(self, add_intercept=True):
        """
        Initializes the class with a boolean indicating whether or not the
        class needs to add a column of 1s to all feature matrices to fit an
        intercept and an empty beta_hat vector that will hold the regression
        model's coefficients.
        
        Parameters
        ----------
        add_intercept : bool, optional
            Tells the class if it needs to add a column of 1s in the first
            column of any data set passed to it, for fitting or prediction. If
            the user does not want to include an intercept in the model, or 
            has already included a column of 1s in the data set for the 
            intercept, this should be set to False. The default is True.

        Returns
        -------
        None.

        """
        self.add_intercept = add_intercept
        self.beta_hat = None
        return
    
    def fit():
        """This method will be overwritten by each of its child classes 
        because the method of fitting the linear model will vary from
        algorithm to algorithm.
        """
        pass
    
    def _predict(self, X):
        """This method predicts the response values of the input array, X, in 
        the scale the model is estimated in; e.g. a logistic model will return
        predictions in log-odds. The columns of X must match the number of 
        columns on the array on which the model was fit. The ordering must be
        identical as well for the predictions to mean anything.

        Parameters
        ----------
        X : numpy ndarray
            A n x m matrix, where the n rows represent observations and the m
            columns represent features of the observations.

        Returns
        -------
        numpy ndarray
            Returns a numpy ndarray with n elements that are the predicted 
            values of the response for each observation in X.

        """
        
        X_copy = self._add_intercept(X)
        
        # Return the predictions.
        return np.matmul(X_copy, self.beta_hat)
    
    def _add_intercept(self, X):
        # If this object needs to add an intercept to new data, add one.
        if self.add_intercept == True:
            # Create an array of 1s equal in length to the observations in X.
            intercept_column = np.repeat(1, repeats=X.shape[0])
            # Insert it at the 0-th column index.
            X_copy = np.insert(X, 0, intercept_column, axis=1)
        # Otherwise, just copy X.
        else:
            X_copy = X
        
        return X_copy
    
class LogisticRegression(LinearModel):
    def __init__(self, add_intercept=True):
        super().__init__()
    
    def _inv_logit(self, X):
        """
        This method takes in a vector of coefficients for a logistic 
        regression model and a matrix of data and returns the probabilities of
        belonging to the class 1 by first calculating the log-odds and 
        translating the log-odds to probabilities. It is used by the 
        _log_likelihood and predict_probabilities methods.

        Parameters
        ----------
        beta : numpy array
            A 1-D vector of coefficients in a logistic regression model.
        X : numpy array
            A 2-D matrix where rows represent observations and columns 
            represent variables.

        Returns
        -------
        probabilities : numpy array
            A 1-D vector of the probabilities associated from the logistic
            regression.

        """
        # Add an intercept if desired.
        X = self._add_intercept(X)
        # Calculate the numerator of the inverse logit transformation.
        numerator = np.exp(np.matmul(X, self.beta_hat))
        # Calculate the denominator of the inverse logit transformation.
        denominator = 1 + np.exp(np.matmul(X, self.beta_hat))
        # Calculate the probabilities.
        probabilities =  numerator / denominator 
        
        return probabilities
    
    def _log_likelihood(self, y, X):
        """
        Overwrites the _log_likelihood method inherited from the RegressorMCMC
        class to calculate the log-likelihood of the logistic regression
        coefficients given binomially-distributed data. It is used in the
        model fitting process.

        Parameters
        ----------
        y : numpy array
            A 1-D vector of 0s and 1s representing the two classes.
        X : numpy array
            A 2-D matrix where rows represent observations and columns 
            represent variables.
        beta : numpy array
            A 1-D vector of coefficients in a logistic regression model.

        Returns
        -------
        log_likelihood : float
            The log-likelihood of the beta vector given the data.

        """
        # Add an intercept if desired.
        X = self._add_intercept(X)
        # Calculate the log-likelihood of beta given the data.
        log_likelihood = np.sum(y*np.log(self._inv_logit(self.beta_hat, X)) 
                               + (1-y)*np.log((1-self._inv_logit(self.beta_hat
                                                                 ,X))))
        
        return log_likelihood
    
    def fit(self, X, y):
        # Add an intercept if desired.
        X = self._add_intercept(X)
        beta_start = np.repeat(0, X.shape[1])
        
        def _log_likelihood_(beta):
            # Calculate the log-likelihood of beta given the data.
            log_likelihood = np.sum(y*np.log(self._inv_logit(beta, X)) 
                                    + (1-y)*np.log((1-self._inv_logit(beta, 
                                                                      X))))
        
            return log_likelihood
        
        def _log_likelihood_optimize(*args):
            beta = np.array(*args)
            return _log_likelihood_(beta)
        
        newton(_log_likelihood_optimize)
        pass
    
    def predict_probabilities(self, X):
        """
        This method returns predictions of belonging to class 1 in 
        probabilities because the predict method will give predictions in 
        log-odds.

        Parameters
        ----------
        X : numpy array
            A 2-D matrix where rows represent observations and columns 
            represent variables.

        Returns
        -------
        predicted_probabilities : numpy array
            A 1-D array of the predicted probabilites of belonging to class 1.

        """
        
        # Add an intercept if desired.
        X = self._add_intercept(X)
        
        # Calculate the probability of each new observation belonging to 
        # class 1.
        predicted_probabilities = self._inv_logit(X)
            
        return predicted_probabilities
    
    def predict_classes(self, X, boundary=0.5):
        """
        This method predicts the class of new observations based on a decision 
        boundary for probability. If predicted probability >= boundary, it is
        predicted to belong to class 1.

        Parameters
        ----------
        X : numpy array
            A 2-D matrix where rows represent observations and columns 
            represent variables.
        boundary : float, optional
            A float on the closed interval between 0 and 1 and is the minimum
            predicted probability to classify a new observation of belonging 
            to class 1. The default is 0.5.

        Returns
        -------
        predicted_classes : numpy array
            DESCRIPTION.

        """
        # Add an intercept if desired.
        X = self._add_intercept(X)
        # Predict the probabilities of belonging to class 1.
        predicted_probabilities = self.predict_probabilities(X)
        # Set predictions to 1 or 0 based on the decision boundary.
        predicted_classes = np.where(predicted_probabilities >= boundary, 1, 0)
        
        return predicted_classes



In [97]:
def _add_intercept(X, add_intercept=True):
        # If this object needs to add an intercept to new data, add one.
        if add_intercept == True:
            # Create an array of 1s equal in length to the observations in X.
            intercept_column = np.repeat(1, repeats=X.shape[0])
            # Insert it at the 0-th column index.
            X_copy = np.insert(X, 0, intercept_column, axis=1)
        # Otherwise, just copy X.
        else:
            X_copy = X
        
        return X_copy

def _sigmoid(beta, X):

        # Calculate the numerator of the inverse logit transformation.
        numerator = 1
        # Calculate the denominator of the inverse logit transformation.
        denominator = 1 + np.exp(-np.dot(X, beta))
        
        return numerator/denominator

def _log_likelihood_(beta, X, y):
    
    p_hat = _sigmoid(beta, X)
    
    # Calculate the log-likelihood of beta given the data.
    log_likelihood = np.sum(y*np.log(p_hat)
                        + (1-y)*(1-np.log(p_hat)))
        
    return log_likelihood
        
def _neg_log_likelihood(beta, X, y):
    return -_log_likelihood_(beta, X, y)

def gradient(beta, X, y):
    p_hat = _sigmoid(beta, X)
    return np.dot(X.T, (p_hat-y))

In [81]:
from sklearn.datasets import make_classification

x, y = make_classification(n_samples=1000, n_features=4,n_informative=2)



ValueError: operands could not be broadcast together with shapes (100,) (1000,) 

In [98]:
from scipy.stats import bernoulli
np.random.seed(1)

cols = 4
n = 100

beta = np.random.normal(size=cols)
x = np.random.normal(size=(n, cols-1))
X = _add_intercept(x)

log_odds = np.matmul(X, beta) #+ np.random.normal(size=n)
probs = 1/(1+np.exp(-log_odds))

my_bernoulli = bernoulli(p=probs)
y = my_bernoulli.rvs()

gradient(beta_start, X, y)

array([-3.84217423, -8.60238116, -3.77242828, 27.7216349 ])

In [102]:

beta_start = np.random.normal(size=X.shape[1])


#calling minimizer with Powell's or BFGS method
minimize(_neg_log_likelihood,
         beta_start,
         args=(X,y),
         jac=gradient,
         method='Newton-CG')#, 
         #options = {'maxiter': 50})
#opt_weights = fmin_tnc(func=cost_function, x0=theta, fprime=gradient, args=(x, y.flatten()))

     fun: -24.06852887865361
     jac: array([-1.41508360e-04, -3.76990044e-05,  2.54210515e-05,  1.05975311e-05])
 message: 'Optimization terminated successfully.'
    nfev: 8
    nhev: 0
     nit: 7
    njev: 23
  status: 0
 success: True
       x: array([ 1.2866222 , -0.215654  ,  0.03967135, -1.10360551])

In [103]:
import statsmodels.api as sm

# building the model and fitting the data
log_reg = sm.Logit(y, X).fit()

print(log_reg.summary())

Optimization terminated successfully.
         Current function value: 0.486228
         Iterations 6
                           Logit Regression Results                           
Dep. Variable:                      y   No. Observations:                  100
Model:                          Logit   Df Residuals:                       96
Method:                           MLE   Df Model:                            3
Date:                Tue, 13 Apr 2021   Pseudo R-squ.:                  0.1664
Time:                        23:12:04   Log-Likelihood:                -48.623
converged:                       True   LL-Null:                       -58.326
Covariance Type:            nonrobust   LLR p-value:                 0.0002253
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.2866      0.283      4.539      0.000       0.731       1.842
x1            -0.2157      0.